In [ ]:
using MeshCat
using RigidBodyDynamics
using MeshCatMechanisms

In [ ]:
vis = Visualizer()
# open(vis)  # open the visualizer in a separate tab/window
IJuliaCell(vis) # render the visualizer here inside the jupyter notebook

In [ ]:
urdf = joinpath(dirname(pathof(MeshCatMechanisms)), "..", "test", "urdf", "Acrobot.urdf")
robot = parse_urdf(Float64, urdf)
delete!(vis)
mvis = MechanismVisualizer(robot, URDFVisuals(urdf), vis)
set_configuration!(mvis, [0.0, 0.0])

In [ ]:
state = MechanismState(robot, randn(2), randn(2))
t, q, v = simulate(state, 5.0);

## Animating a Trajectory

You can animate a trajectory (a list of times and configurations) with the `setanimation!()` function. To create an animation and play it in the visualizer, you just need to do:

In [ ]:
setanimation!(mvis, t, q)

### Replaying the Animation

When you use `setanimation!`, the entire animation is sent to the visualizer, so you can replay it at will. In the viewer, click "Open Controls" in the top right corner, go to "Animation" -> "default" and click "play" to play the animation again. 

### Recording the Animation

You can also record an animation directly from the MeshCat viewer. Just open the controls and go to "Animation" -> "default" -> "Recording" and click "record". The animation will play back (slowly, as it needs to cleanly capture every frame), and then you'll be able to download the resulting frames. 

Currently, MeshCat downloads an animation as a `.tar` file containing all of the individual frames. To convert that into a video, you just need to install the `ffmpeg` program (`apt-get install ffmpeg` on Ubuntu 18.04) and then run: 

```julia
MeshCat.convert_frames_to_video("/path/to/downloads/meshcat_xxx.tar")
```

### Visualizing Frames and Points

You can use the lower-level `setelement!()` function to attach geometries to frames in the mechanism. For example, to visualize a coordinate frame with a triad, you can do: 

In [ ]:
lower_arm = bodies(robot)[end]
body_frame = default_frame(lower_arm)
setelement!(mvis, body_frame)

And likewise, to visualize a point attached to a frame, you can do:

In [ ]:
radius = 0.05
name = "my_point"
setelement!(mvis, Point3D(body_frame, 0.2, 0.2, 0.2), radius, name)

The triad and point are included in the MeshCat scene tree, so each will move along with the body to which it's attached:

In [ ]:
setanimation!(mvis, t, q)

In [ ]:
using ValkyrieRobot

val = Valkyrie();
delete!(vis)
mvis = MechanismVisualizer(
    val.mechanism, 
    URDFVisuals(ValkyrieRobot.urdfpath(), package_path=[dirname(dirname(ValkyrieRobot.urdfpath()))]),
    vis);

In [ ]:
using ValkyrieRobot.BipedControlUtil: Side, flipsign_if_right

function initialize!(state::MechanismState, val::Valkyrie)
    zero!(state)
    mechanism = val.mechanism
    for side in instances(Side)
        set_configuration!(state, findjoint(mechanism, "$(side)KneePitch"), [1.205])
        set_configuration!(state, findjoint(mechanism, "$(side)HipPitch"), [-0.49])
        set_configuration!(state, findjoint(mechanism, "$(side)AnklePitch"), [-0.71])
        set_configuration!(state, findjoint(mechanism, "$(side)ShoulderPitch"), [0.300196631343025])
        set_configuration!(state, findjoint(mechanism, "$(side)ShoulderRoll"), [flipsign_if_right(-1.25, side)])
        set_configuration!(state, findjoint(mechanism, "$(side)ElbowPitch"), [flipsign_if_right(-0.785398163397448, side)])
        set_configuration!(state, findjoint(mechanism, "$(side)ForearmYaw"), [1.571])
    end
    set_configuration!(state, val.basejoint, [1; 0; 0; 0; 0; 0; 1.025])
    nothing
end
state = MechanismState(val.mechanism)
initialize!(state, val)
set_configuration!(mvis, configuration(state))

In [ ]:
# Handling robots with fixed joints
urdf = joinpath(dirname(pathof(MeshCatMechanisms)), "..", "test", "urdf", "Acrobot.urdf")
robot = parse_urdf(Float64, urdf)
RigidBodyDynamics.remove_fixed_tree_joints!(robot)
vis = MechanismVisualizer(robot, URDFVisuals(urdf))

# with the elbow fixed, there is only one configuration element
set_configuration!(vis, [0.5]) 
IJuliaCell(vis)